## Lưu ý: Tạo file `raw_data.xlsx` với sheet = `country` trước khi chạy code

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from datetime import date,timedelta
import time

In [2]:
url = f"https://www.worldometers.info/coronavirus/"
browser = webdriver.Chrome(executable_path="Driver/chromedriver.exe")
browser.get(url)

In [3]:
html_text = browser.page_source
tree = BeautifulSoup(html_text, 'html.parser')
tbody = tree.find("tbody")
trows = tbody.find_all("tr",{"role": "row"})
total_row_world = tbody.find_all("tr",{"class" : "total_row_world"})
list_data_country = [trow.text.split("\n")[1:-2] for trow in trows if trow not in total_row_world]

In [4]:
list_data_continent = []
for trow in total_row_world:
    if "World" not in trow.text:
        temp_data = trow.text.split("\n")[3:-7]
        temp_data[1] = temp_data[0]
        list_data_continent.append(temp_data[1:])

In [5]:
list_data_country

[['1',
  'USA',
  '32,824,389',
  '',
  '586,152 ',
  '',
  '25,379,831',
  '6,858,406',
  '9,912',
  '98,695',
  '1,762',
  '439,643,492',
  '1,321,898',
  '332,584,983 '],
 ['2',
  'India',
  '17,306,300',
  '',
  '195,116 ',
  '',
  '14,296,640',
  '2,814,544',
  '8,944',
  '12,441',
  '140',
  '277,918,810',
  '199,791',
  '1,391,049,556 '],
 ['3',
  'Brazil',
  '14,340,787',
  '',
  '390,925 ',
  '',
  '12,809,169',
  '1,140,693',
  '8,318',
  '67,079',
  '1,829',
  '43,538,104',
  '203,648',
  '213,790,656 '],
 ['4',
  'France',
  '5,498,044',
  '',
  '102,858 ',
  '',
  '4,363,098',
  '1,032,088',
  '5,978',
  '84,079',
  '1,573',
  '74,505,035',
  '1,139,371',
  '65,391,401 '],
 ['5',
  'Russia',
  '4,762,569',
  '',
  '108,232 ',
  '',
  '4,388,008',
  '266,329',
  '2,300',
  '32,624',
  '741',
  '127,800,000',
  '875,429',
  '145,985,566 '],
 ['6',
  'Turkey',
  '4,629,969',
  '',
  '38,358 ',
  '',
  '4,073,644',
  '517,967',
  '3,590',
  '54,421',
  '451',
  '45,884,258',
 

In [6]:
df_country = pd.DataFrame(list_data_country, columns = ["#", "Country, Other", "Total Cases", "New Cases", "Total Deaths", 
                                        "New Death", "Total Recovered", "Active Case", "Serious Critical", 
                                        "Tot Cases/1M pop", "Death/1M pop","Total test","Test/1M pop", "Population"])
df_country.set_index("#", inplace=True)
df_country["Time"] = date.today() - timedelta(1)
#df_country = df_country.replace([""," "], "0")
df_country.head()

,"Country, Other",Total Cases,New Cases,Total Deaths,New Death,Total Recovered,Active Case,Serious Critical,Tot Cases/1M pop,Death/1M pop,Total test,Test/1M pop,Population,Time
#,,,,,,,,,,,,,,
1,USA,"32,824,389",,"586,152",,"25,379,831","6,858,406","9,912","98,695","1,762","439,643,492","1,321,898","332,584,983",2021-04-25
2,India,"17,306,300",,"195,116",,"14,296,640","2,814,544","8,944","12,441",140,"277,918,810","199,791","1,391,049,556",2021-04-25
3,Brazil,"14,340,787",,"390,925",,"12,809,169","1,140,693","8,318","67,079","1,829","43,538,104","203,648","213,790,656",2021-04-25
4,France,"5,498,044",,"102,858",,"4,363,098","1,032,088","5,978","84,079","1,573","74,505,035","1,139,371","65,391,401",2021-04-25
5,Russia,"4,762,569",,"108,232",,"4,388,008","266,329","2,300","32,624",741,"127,800,000","875,429","145,985,566",2021-04-25


In [7]:
writer = pd.ExcelWriter("raw_data.xlsx", engine='xlsxwriter',datetime_format = "YYYY-MM-DD")

In [8]:
old_df_country = pd.read_excel(writer, sheet_name= "country", index_col=0)
frames = [old_df_country, df_country]
result = pd.concat(frames)
result.to_excel(writer, sheet_name= "country")
writer.save()